# **Projeto: Percolação por sítios em rede quadrada**
### **Programação Estatística - Projeto de Fim de Curso**

- Docente: Rafael Izbicki
- Discentes:
    - Andressa Nascimento dos Santos - RA
    - Nicolas Magalhães Santana e Silva - RA 830225

---

# **Introdução**

A teoria da percolação é um modelo fundamental para descrever fenômenos de conexão em sistemas desordenados, como o fluxo de l ́ıquidos em meios porosos, a propagação de incêndios florestais ou a condutividade em materiais compósitos.

O objetivo deste projeto é estimar, por meio de simulação de Monte Carlo, o limiar crítico de percolação $p_c$ em uma grade quadrada bidimensional e analisar o comportamento do sistema próximo a esse ponto crítico.

As principais questões são:
- Qual o valor de $p$ a partir do qual um "caminho" de ponta a ponta na grade se torna provável?
- Como essa probabilidade varia com o tamanho da grade ($N$)?
- Qual a distribuição de tamanho do maior aglomerado _(cluster)_ de sítios conectados, especialmente no limiar crítico?

---

# **Descrição do modelo gerador de dados**

O modelo é uma grade (ou reticulado) quadrada $\mathcal{L}$ de dimensões $N \times X$. Cada sítio $(i, j) \in \mathcal{L}$ pode estar em um de dois estados: "ocupado" ou "vazio". A ocupação de cada sítio é um evento independente de Bernoulli com probabilidade $p$, ou seja, $P(\text{sítio }(i, j)\text{ ocupado})=p$.

Um _cluster_ é definido como um conjunto de sítios ocupados adjacentes (vizinhos de von Neumann: acima, abaixo à esquerda, à direita). 

Diz-se que o sistema "percola" se existir um cluster que conecte a borda superior à borda inferior da grade (ou a esquerda à direita).

A probabilidade de percolação, $\theta(p,N)$, é a probabilidade de que uma grade $N \times N$ gerada com parâmetro $p$ percole.

O limiar crítico $p_c$ é definido no limite $N \rightarrow \infty$ como o valor de $p$ em que $\theta(p)$ transita de $0$ para $1$.

---

# **Metodologia da simulação**

A simulação seguirá os seguintes passos para um dado par $(N, p)$:
1. **Geração da Grade:** Crie uma matriz $N \times N$ e preencha cada célula com $1$ (ocupado) com probabilidade $p$ e $$0$ (vazio) com probabilidade $1-p$.

2. **Verificação de Percolação:** Verifique se, na configuração gerada, é possível ir da borda inferior para a superior.

3. **Estimação de $p_c(N)$**: Para um dado $N$, a probabilidade de percolação $\theta(p, N)$ será estimada pela frequência de percolação em um grande número de simulações. O limiar finito, $p_c(N)$, é tipicamente definido como o valor de $p$ para o qual $θ(p, N) \approx \frac{1}{2}$. Isso pode ser encontrado varrendo $p$ em uma malha fina ou usando um método de busca (e.g., busca binária) para encontrar o ponto de cruzamento. A extrapolação para o limite $N \rightarrow \infty$ pode ser investigada através da teoria de escala de tamanho finito, plotando $p_c(N)$ versus $1/N$ e analisando o comportamento assintótico.

### **Primeiro algoritmo: Simulação de percolação**
Nesse algoritmo, realizamos os passos 1 (Geração de grade) e 2 (Verificação de percolação)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
np.random.seed(42)

In [19]:
np.random.seed(42)
def simulacao_percolacao(N, p):
  ''' Recebe os parâmetros N (o tamanho da matriz será NxN) e p (probabilidade da entrada da matriz ser 1, e 0 caso contrário),
retorna se a matriz percolou, o número de clusters e o tamanho do maior cluster'''
  percolou = False
  maior_cluster = 0
  num_clusters = 0

  grade = np.random.rand(N, N) < p
  grade = np.where(grade, 1, 0)
  if not np.any(grade):
    return False, 0, 0
  # Se a matriz for inteiramente de zeros, já retorna que não percola e não teve clusters

  estrutura = ndimage.generate_binary_structure(rank=2, connectivity=1)
  # Define a estrutura de Vizinhos de von Neumann

  grade_clusters, num_clusters = ndimage.label(grade, estrutura)
  # Função de etiquetar as matrizes em clusters
  coluna_1 = grade_clusters[:, 0]
  coluna_n = grade_clusters[:, -1]
  linha_1 = grade_clusters[0]
  linha_n = grade_clusters[-1]

  intersecoes_vertical = np.intersect1d(linha_1, linha_n)
  intersecoes_horizontal = np.intersect1d(coluna_1, coluna_n)

  percolou_vertical = np.any(intersecoes_vertical > 0)
  percolou_horizontal = np.any(intersecoes_horizontal > 0)
  if percolou_vertical or percolou_horizontal:
    percolou = True

  grade_clusters_d1 = np.ravel(grade_clusters)

  contagem = np.bincount(grade_clusters_d1)
  # Conta ocupação de cada etiqueta de cluster
  maior_cluster = max(contagem[1:])
  # Maior cluster é o elemento que ocupou mais sítios (excluindo o zero)

  return percolou, maior_cluster, num_clusters

perc, maior, num = simulacao_percolacao(8, 0.5)
print(f"O sistema percolou? {perc}")
print(f"Qual o tamanho do maior cluster? {maior}")
print(f"Qual o número total de clusters? {num}")

O sistema percolou? False
Qual o tamanho do maior cluster? 15
Qual o número total de clusters? 6


### **Segundo algoritmo: **

---

# **Plano de experimento e métricas**

- **Tamanhos da grade:** Realize simula¸c˜oes para N ∈ {64, 128, 256, 512}.

- **Varredura de p:** Para cada N , execute simula¸c˜oes para uma faixa de valores p.
Para cada par (N, p), realize pelo menos M = 500 r´eplicas.

- **M´etricas a coletar:** Para cada simula¸c˜ao, registre: (a) se houve percola¸c˜ao
(booleano), (b) o tamanho do maior cluster, (c) o n´umero total de clusters.

- **An´alise do maior cluster:** No valor estimado ˆpc(N ), colete a distribui¸c˜ao emp´ırica
do tamanho do maior cluster em m´ultiplas simula¸c˜oes.

---

# **Resultados**